# SQL 작성

In [1]:
import sys
import os
import pandas as pd
import cv2
import xml.etree.ElementTree as ET
sys.path.append('../../src')
import DISData as DD

## SQL 연결

In [2]:
doUT = DD.SQL()

host:  192.168.0.201
user:  diadmin
password:  ········


SQL connection done


## query 작성

In [3]:
query1 = (f'''CREATE TABLE IF NOT EXISTS robo_Anti_spoof_info (
        id INT(11) NOT NULL AUTO_INCREMENT primary key,
        img_dir VARCHAR(300) NOT NULL,
        img_format VARCHAR(10) NOT NULL,
        img_width INT(11) NOT NULL,
        img_height INT(11) NOT NULL,
        color_space VARCHAR(10) NOT NULL,
        label_dir VARCHAR(300) NOT NULL,
        label_format VARCHAR(10) NOT NULL,
        label VARCHAR(10) NOT NULL,
        label_id INT(10) NOT NULL,
        box_xtl INT(10) NOT NULL,
        box_ytl INT(10) NOT NULL,
        box_w INT(10) NOT NULL,
        box_h INT(10) NOT NULL)''')

In [4]:
query2 = (f'''INSERT INTO robo_Anti_spoof_info
        (img_dir, img_format, img_width, img_height, color_space,
        label_dir, label_format, label, label_id, box_xtl, 
        box_ytl, box_w, box_h) VALUES 
          (%s, %s, %s, %s, %s,  %s, %s, %s, %s, %s,  %s, %s, %s)''')

## qurey 실행

In [5]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [6]:
label_path = []
label_name= []

# label 경로
label_path = 'Z:\\robo_Anti_spoof\\data\\RGB\\labeldata'
image_path = 'Z:\\robo_Anti_spoof\\data\\RGB\\rawdata'

value_list1 = []
error_list = []
count = 0

labels = os.listdir(label_path)
for num, label in enumerate(labels):
    total_cnt = len(labels)
    color_space = 'RGB'
    # print(label)
    label_ids = os.listdir(os.path.join(label_path,label))
    for label_id in label_ids[1:]:
        label_files = os.listdir(os.path.join(label_path, label, label_id))
        
        for cnt, label_file in enumerate(label_files):
            label_dir = os.path.join(label_path,label,label_id,label_file)
            # print(label_dir)

            label_name, label_format = os.path.splitext(os.path.basename(label_file))
            label_format = label_format.replace('.','')
            
            tree = ET.parse(label_dir)
            root = tree.getroot()
            
            img_dir = label_dir.replace('label','raw').replace('.xml','.jpg')
            
            

            img_file = os.path.basename(img_dir)
            img_name = label_name
            img_format = 'jpg'

            if os.path.exists(img_dir) and os.path.getsize(img_dir) != 0:
                img = cv2.imread(img_dir)            
            else:
                error_list.append(img_dir)
                continue
            img_w = img.shape[0]
            img_h = img.shape[1]

            img_dir = img_dir.replace('Z:','DataBase')
            label_dir = label_dir.replace('Z:','DataBase')
            
            for i in root.findall('object'):
                boxes = i.find('bndbox')
                box_xtl = int(boxes.find('xmin').text)
                box_ytl = int(boxes.find('ymin').text)
                box_xmax = int(boxes.find('xmax').text)
                box_ymax = int(boxes.find('ymax').text)
                box_w = abs(box_xtl - box_xmax)
                box_h = abs(box_ytl - box_ymax)
                

                values = (img_dir, img_format, img_w, img_h, color_space,
                          label_dir, label_format, label, label_id, box_xtl, 
                          box_ytl, box_w, box_h)

                value_list1.append(values)
            

        # query 실행
            if len(value_list1) >= 300:
                print(value_list1[-1])
                doUT.insert_dataset_values(query2, value_list1)
                value_list1 = []
                print(num, total_cnt)

doUT.insert_dataset_values(query2, value_list1)
value_list1 = []
print('done')

('DataBase\\robo_Anti_spoof\\data\\RGB\\rawdata\\live\\11\\real_11_20_jpg.rf.6090a980c933645eb26d7f3e2f1d43fd.jpg', 'jpg', 602, 602, 'RGB', 'DataBase\\robo_Anti_spoof\\data\\RGB\\labeldata\\live\\11\\real_11_20_jpg.rf.6090a980c933645eb26d7f3e2f1d43fd.xml', 'xml', 'live', '11', 113, 1, 346, 471)
insert 300 done
0 2
('DataBase\\robo_Anti_spoof\\data\\RGB\\rawdata\\live\\11\\real_11_77_jpg.rf.f7d4051a58eff5ff076d3affd908ecfd.jpg', 'jpg', 640, 640, 'RGB', 'DataBase\\robo_Anti_spoof\\data\\RGB\\labeldata\\live\\11\\real_11_77_jpg.rf.f7d4051a58eff5ff076d3affd908ecfd.xml', 'xml', 'live', '11', 160, 148, 297, 421)
insert 300 done
0 2
('DataBase\\robo_Anti_spoof\\data\\RGB\\rawdata\\live\\12\\real_12_413_jpg.rf.bd515b96d30136568f5e5f9b290f38a6.jpg', 'jpg', 606, 606, 'RGB', 'DataBase\\robo_Anti_spoof\\data\\RGB\\labeldata\\live\\12\\real_12_413_jpg.rf.bd515b96d30136568f5e5f9b290f38a6.xml', 'xml', 'live', '12', 199, 93, 249, 352)
insert 300 done
0 2
('DataBase\\robo_Anti_spoof\\data\\RGB\\rawdata